- trf
    - ca37: distance=passthrough
    - fa15: distance=pwr
- imp
    - 2474: tune=False
    - 20cf: tune=True
- clf
    - 2c44: ensemble=True, time=20
    - 25f2: ensemble=False, time=20

In [24]:
dct = {'a':5}
list(dct.values())[:2]

[5]

In [1]:
from LiveAMP import *
# Flags().run()
# self = run_amp(135)

In [22]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/135/Y/n/_allcrse/202308/fa15/1bfb/2f70.pkl'
self = MyBaseClass().load(fn)
S = self.Y.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
    'actual': y['actual'].sum(),
    # 'predicted': y['predicted'].sum(),
    'predicted': y['proba'].sum(),
    'train_score': self.train_score,
    'test_score': log_loss(y['actual'], y['proba'], labels=[False,True], sample_weight=1+(2*y['actual']-1)*self.weight) * 100,
    'weight': self.weight,
})).prep()
proj_mask = S.eval(f"pred_code==@self.proj_code")
proj_col = f'{self.proj_code}_projection'
S = (
    S[~proj_mask]
    .join(S[proj_mask]['predicted'].droplevel('pred_code').rename(proj_col))
    .join(self.mlt.squeeze().rename('mlt'))
)
for k in ['predicted','actual',proj_col]:
    S[k] *= S['mlt']
alpha = 1
S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
S['error'] = S['predicted'] - S['actual']
S['error_pct'] = S['error'] / S['actual'] * 100
S[[proj_col,'overall_score','error_pct','error','predicted','actual','test_score','train_score']]
# S

202408_projection  \
crse_code levl_code styp_code train_code pred_code trf_hash imp_hash clf_hash sim                      
_allcrse  ug        n         202308     202108    fa15     1bfb     2f70     0          2894.712821   
                                                                              1          2893.816003   
                                                                              2          2894.077923   
                                                                              3          2894.350183   
                                                                              4          2895.152572   
                                                                              5          2895.243542   
                                                                              6          2893.920443   
                                                                              7          2894.078815   
                                                                              8          2895.745709   
                                                                              9          2895.310526   
                                         202208    fa15     1bfb     2f70     0          2851.006355   
                                                                              1          2850.123078   
                                                                              2          2850.381044   
                                                                              3          2850.649193   
                                                                              4          2851.439467   
                                                                              5          2851.529064   
                                                                              6          2850.225942   
                                                                              7          2850.381922   
                                                                              8          2852.023648   
                                                                              9          2851.595036   
                                         202308    fa15     1bfb     2f70     0          2751.741486   
                                                                              1          2750.888963   
                                                                              2          2751.137947   
                                                                              3           2751.39676   
                                                                              4          2752.159518   
                                                                              5          2752.245995   
                                                                              6          2750.988245   
                                                                              7          2751.138794   
                                                                              8          2752.723359   
                                                                              9           2752.30967   

                                                                                   overall_score  \
crse_code levl_code styp_code train_code pred_code trf_hash imp_hash clf_hash sim                  
_allcrse  ug        n         202308     202108    fa15     1bfb     2f70     0        26.022499   
                                                                              1         26.03475   
                                                                              2        26.037411   
                                                                              3        26.034631   
                                                                              4        26.053472   
                                                      

In [ ]:
            S = self.Y.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
                'actual': y['actual'].sum(),
                # 'predicted': y['predicted'].sum(),
                'predicted': y['proba'].sum(),
                'train_score': self.train_score,
                'test_score': log_loss(y['actual'], y['proba'], labels=[False,True], sample_weight=1+(2*y['actual']-1)*self.weight) * 100,
                # 'test_score': (1 - f1_score(y['actual'], y['predicted'], sample_weight=1+(2*y['actual']-1)*clf.weight))*100,
                'weight': self.weight,
            })).prep()


In [ ]:
read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/135/X.pkl')['mlt']
# self.mlt

In [ ]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/135/X_proc/n/fa15/1bfb.pkl'
self = MyBaseClass().load(fn)
Z = pd.concat(self.X_proc).join(Y).fillna({c:0 for c in cols}).addlevels({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]}).prep(bool=True, cat=True)

In [ ]:
A = Z.query(f"pred_code=={self.train_code}").copy()
t = A.query(f"sim==0").groupby([self.crse_code,'__coll_code','__remote'], observed=True).sample(frac=0.7)
mask = A.reset_index()['index'].isin(t.reset_index()['index']).values
t.shape, mask.shape
# A['mask'] = mask.values
# A['mask'].mean()
# mask = A.assign(mask=A.reset_index()['index'].isin(t)).pop('mask')
# mask.mean(), mask.shape
    

In [ ]:
def f(X):
    

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/135/X_proc/n/fa15/1bfb.pkl'
self = MyBaseClass().load(fn)
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
Y = self.Y.query(f"crse_code.isin({cols})").unstack().droplevel(0,1)
Z = pd.concat(self.X_proc).join(Y).fillna({c:0 for c in cols}).addlevels({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]}).prep(bool=True, cat=True)


In [ ]:
hist = 4

clf_dct = param_dcts['clf'][0][2]
X = Z.query(f"pred_code=={self.train_code}").copy()
t = X.query(f"sim==0").groupby([self.crse_code,'__coll_code','__remote'], observed=True)
# mask = X.reset_index()['index'].isin(t.sample(frac=0.75).reset_index()['index']).values
y = X.pop(self.crse_code)
weight = 0
# weight_hist = {weight: np.inf}
L = []
for i in range(20):
    mask = X.reset_index()['index'].isin(t.sample(frac=0.75).reset_index()['index']).values
    dct = clf_dct | {
        'X_train':X[mask],
        'y_train':y[mask],
        'X_val':X[~mask],
        'y_val':y[~mask],
        'sample_weight':1+(2*y[mask]-1)*weight,
        'sample_weight_val':1+(2*y[~mask]-1)*weight,
        'log_file_name': self.path.with_stem(f"{self.path.stem}_{i}").with_suffix('.log'),
        'task':'classification',
        'verbose':0,
        
        'log_type': 'all',
        # 'time_budget': 60*10,
    }
    clf = fl.AutoML(**dct)
    clf.fit(**dct)
    X_all = Z.copy()
    Y = X_all.pop(self.crse_code).rename('actual').to_frame().assign(proba=clf.predict_proba(X_all)[:,1]).prep(bool=True).copy()
    S = Y.groupby('pred_code').sum()
    S['err'] = S['proba'] / S['actual'] - 1
    S['proba'] *= S['actual'] > 0
    T = S.query(f"pred_code!={self.proj_code}").sum()
    T['err'] = T['proba'] / T['actual'] - 1
    print(weight, T['err'])
    S.disp(10)
    T.to_frame().T.disp(10)
    weight = np.clip(weight - T['err'], -1, 1)
    L.append(T['err'])
    if len(L) >= hist:
        s = np.std(L[-hist:])
        print(i, s)
        if s < 0.005:
            break
        

# weight_hist[weight] = abs(T['err'])

# weight = min(weight_hist, key=weight_hist.get)
# print(weight, weight_hist[weight])

In [ ]:
clf_dct = param_dcts['clf'][0][2]
X = Z.query(f"pred_code=={self.train_code}").copy()
t = X.query(f"sim==0").groupby([self.crse_code,'__coll_code','__remote'], observed=True)
mask = X.reset_index()['index'].isin(t.sample(frac=0.75).reset_index()['index']).values
y = X.pop(self.crse_code)
weight = 0
# weight_hist = {weight: np.inf}
L = []
for i in range(10):
# mask = X.reset_index()['index'].isin(t.sample(frac=0.75).reset_index()['index']).values
dct = clf_dct | {
'X_train':X[mask],
'y_train':y[mask],
'X_val':X[~mask],
'y_val':y[~mask],
'sample_weight':1+(2*y[mask]-1)*weight,
'sample_weight_val':1+(2*y[~mask]-1)*weight,
'task':'classification',
'verbose':0,
'log_file_name': self.path.with_stem(f"{self.path.stem}_{i}").with_suffix('.log'),
'log_type': 'all',
# 'time_budget': 60*10,
}
clf = fl.AutoML(**dct)
clf.fit(**dct)
X_all = Z.copy()
Y = X_all.pop(self.crse_code).rename('actual').to_frame().assign(proba=clf.predict_proba(X_all)[:,1]).prep(bool=True).copy()
S = Y.groupby('pred_code').sum()
S['err'] = S['proba'] / S['actual'] - 1
S['proba'] *= S['actual'] > 0
T = S.query(f"pred_code!={self.proj_code}").sum()
T['err'] = T['proba'] / T['actual'] - 1
print(weight, T['err'])
S.disp(10)
T.to_frame().T.disp(10)
L.append(abs(T['err']))
if len(L)>=5
s = np.std(L[-5:])
print(s)

# weight_hist[weight] = abs(T['err'])
weight = np.clip(weight - T['err'], -1, 1)
weight = min(weight_hist, key=weight_hist.get)
print(weight, weight_hist[weight])


In [ ]:
L = [1]
L[-5:]

In [ ]:
S = S.query(f"pred_code!={self.proj_code}")
T = S.sum()
T['proba'] / T['actual'] - 1

In [ ]:
dct['log_file_name']

In [ ]:
clf.fit?

In [ ]:
param_dcts['clf'][0][2]

In [ ]:
self.path.with_stem(f"{self.path.stem}_1").with_suffix('.log')

In [ ]:
X = z.query(f"pred_code=={self.train_code}").copy()
g = X.groupby([self.crse_code,'__coll_code'], observed=True).ngroup()
y = X.pop(self.crse_code)
dct = (clf_dct
        | dict(zip(['X_train', 'X_val', 'y_train', 'y_val'], train_test_split(X, y, train_size=0.75, stratify=g)))
        | {'task':'classification', 'verbose':0, 'log_file_name': self.path.with_stem(f"{self.path.stem}_{i}").with_suffix('.log')}
)
clf = fl.AutoML(**dct)
clf.fit(**dct)


In [ ]:
from sklearn.model_selection import train_test_split
# clf_dct = self.param['clf'][2] | {'task':'classification', 'verbose':0, 'log_file_name': self.path.with_suffix('.log')}
# clf_dct = param_dcts['clf'][0][2] | {'task':'classification', 'verbose':0, 'log_file_name': self.path.with_suffix('.log')}
print(clf_dct)
for i, z in Z.groupby('sim'):
    X = z.query(f"pred_code=={self.train_code}").copy()
    g = X.groupby([self.crse_code,'__coll_code'], observed=True).ngroup()
    y = X.pop(self.crse_code)
    dct = (clf_dct
           | dict(zip(['X_train', 'X_val', 'y_train', 'y_val'], train_test_split(X, y, train_size=0.75, stratify=g)))
           | {'task':'classification', 'verbose':0, 'log_file_name': self.path.with_stem(f"{self.path.stem}_{i}").with_suffix('.log')}
    )
    clf = fl.AutoML(**dct)
    clf.fit(**dct)

    X = z.copy()
    Y = X.pop(self.crse_code).rename('actual').to_frame().assign(proba=clf.predict_proba(X)[:,1]).prep(bool=True).copy()
    break

# A['trn'].mean()



In [ ]:
Y.groupby('pred_code').sum()

In [ ]:
d['y_train']

In [ ]:
clf.fit?

In [ ]:
Z = (
    pd.concat([
    X
    for X in self.X_proc])
    .join(Y)
    
    .fillna({c:0 for c in cols})
    
    .addlevels({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
    
    .prep(bool=True, cat=True)
)


In [ ]:
Z = (
    pd.concat([X.join(Y) for X in self.X_proc])
    .fillna({c:0 for c in cols})
    .addlevels({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
    .prep(bool=True, cat=True)
)


In [ ]:
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
Y = self.Y.query(f"crse_code.isin({cols})").unstack().droplevel(0,1)
Z = [
    X.join(Y)
    .fillna({c:0 for c in cols})
    .addlevels({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
    .prep(bool=True, cat=True)
    for X in self.X_proc]

In [ ]:
A = Z[0].copy()
B = A.groupby([self.crse_code,'__coll_code','__remote'], observed=True).sample(frac=0.7)
# A['mask'] = A.index.isin(B.index)
mask = A.assign(mask=A.index.isin(B.index)).pop('mask')

# mask#.disp()
# A.disp(2)
# A['trn'].mean()
# A.assign(trn = A.reset_index()['index'].isin(B)).mean()

# A
# A.reset_index()['index']
# C = A.join(B)
# A.shape, B.shape, C.shape, C['trn'].mean()
# .reset_index()['index']
# A['tst'] = A.reset_index()['index'].isin(B)
# A.disp()
# B
# A[B]
# mask = A.index.isin(B.index)
# mask.mean()
# mask
# A.shape, B.shape
# transform(lambda x:x.cumcount()).disp()
# A.disp()

In [ ]:
 606/544, 678/606, 2478/2142, 2700/2478, 544/2142, 606/2478, 678/2700, 792/3276, 763/3320, 593/2845

In [ ]:
0.9**15

In [ ]:
from LiveAMP import *
# Flags().run()
self = run_amp(137)

In [ ]:
from LiveAMP import *
# Flags().run()
# self = run_amp(137)
dct = get_stack(136)
# dct[' 50%'].query("clf_hash=='a846' & imp_hash=='1bfb' & trf_hash=='fa15'").sort_index()
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y/n/biol1406/202308/fa15/1bfb/e698.pkl'
# self = MyBaseClass().load(fn)

In [ ]:
path = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/136')
for A in (path / 'Y').iterdir():
    for B in A.iterdir():
        for C in B.iterdir():
            for D in C.iterdir():
                for E in D.iterdir():
                    for F in E.iterdir():
                        if F.suffix == '.pkl':
                            print(F)
                            A = MyBaseClass().load(F)
                            print(A.__dict__.keys())
                            for k in ['Y', 'summary']:
                                A[k] = pd.concat([getattr(clf, k) for clf in A.clf])
                            A.dump(F)


In [ ]:
deepcopy(self.clf[0])

In [ ]:
param_dcts['clf'][0][-1]

In [ ]:
from sklearn.calibration import calibration_curve, CalibrationDisplay, CalibratedClassifierCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import brier_score_loss, classification_report


X, y, w = self.clf[0].X, self.clf[0].Y['actual'], self.clf[0].weight
# clf = deepcopy(self.clf[0])
# X, y, w = clf.X, clf.Y['actual'], clf.weight
clf_dct = param_dcts['clf'][0][-1].copy() | {'split_type':'stratified', 'task':'classification', 'verbose':0}
clf_dct.pop('n_calibrations')
mask = X.eval(f"pred_code!={self.train_code}")

def run(mod, dct=dict()):
    mod.fit(X[~mask], y[~mask], **dct)
    # mod.fit(X[~mask], y[~mask], sample_weight=1+(2*y[~mask]-1)*w, **dct)
    Y = y.rename('actual').to_frame().assign(predicted=mod.predict(X), proba=mod.predict_proba(X)[:,1]).prep(bool=True).copy()
    S = Y.groupby('pred_code').sum().query(f"pred_code!={self.proj_code}")
    S['pred_err'] = (S['predicted'] / S['actual'] - 1) * 100
    S['proba_err'] = (S['proba'] / S['actual'] - 1) * 100
    S.disp(10)
    print(brier_score_loss(Y[mask]['actual'], Y[mask]['proba']))
    print(classification_report(Y[mask]['actual'], Y[mask]['predicted']))
    CalibrationDisplay.from_predictions(Y[mask]['actual'], Y[mask]['proba'])
    # CalibrationDisplay.from_estimator(mod, X[~mask], y[~mask])#, n_bins=100)
    # CalibrationDisplay.from_estimator(mod, X[~mask], y[~mask])#, n_bins=100)
    plt.show()

# base_clf = HistGradientBoostingClassifier(categorical_features="from_dtype")
clf = fl.AutoML(**clf_dct)
run(clf, clf_dct)
clf = CalibratedClassifierCV(fl.AutoML(**clf_dct), method='sigmoid')
run(clf, clf_dct)
clf = CalibratedClassifierCV(fl.AutoML(**clf_dct), method='isotonic')
run(clf, clf_dct)

In [ ]:
mod = cal_clf
Y = y.rename('actual').to_frame().assign(predicted=mod.predict(X), proba=mod.predict_proba(X)[:,1]).prep(bool=True).copy()
S = Y.groupby('pred_code').sum()#.query(f"pred_code!={self.proj_code}")
S['err'] = (S['proba'] / S['actual'] - 1) * 100
S.disp(10)

# cal_clf.predict_proba(X)[:,1].sum()

In [ ]:
640*1.126294, 640*1.068783, 640*1.056075

In [ ]:
read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/X.pkl')['mlt'].query("crse_code=='biol1406'")

In [ ]:
# dct[' 50%']
dct.keys()

In [ ]:
[len(v) for k,v in param_dcts.items()]

In [ ]:
dct['summary'].reset_index().disp(1000)

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/X_proc/n/ca37/857f.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/_allcrse/202308/ca37/857f/c0cb.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/_allcrse/202308/ca37/857f/c0cb.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/_allcrse/202308/fa15/1bfb/a846.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/_allcrse/202308/fa15/1bfb/a846.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/X.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/biol1406/202308/fa15/1bfb/a846.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/136/Y/n/_allcrse/202108/fa15/1bfb/aa8b.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/136/Y/n/biol1406/202108/fa15/1bfb/aa8b.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/136/Y/n/biol1406/202208/fa15/1bfb/aa8b.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/136/Y/n/arts1301/202108/fa15/1bfb/aa8b.pkl'
self = MyBaseClass().load(fn)
self.__dict__.keys()
# dct = read(fn)
# dct.keys()

In [ ]:
# for k,v in self.clf[0].__dict__.items():
size = lambda x: sys.getsizeof(dill.dumps(x))/1000
for k,v in self.clf[0].__dict__.items():
    print(k, type(v), size(v))
# __dict__.keys()
# sys.getsizeof(self)
# w = dill.dumps(self)
# sys.getsizeof(w)/1000000

In [ ]:
e = self.clf[0]._trained_estimator
self.clf[0].predict??
# f = e.estimator
# f.
# self.clf[0].model.estimator
# size(self.clf[0]._trained_estimator)
# size(self.clf[0].model.estimator)
# self.clf[0].model._model

In [ ]:
def f(x):
    print('hi')
y = f(3)
if y != 'fail':
    print('there')

In [ ]:
self.clf[0].__dict__.keys()

In [ ]:
for k in ['Y', 'summary']:
    self[k] = pd.concat([getattr(clf, k) for clf in self.clf])
self.__dict__.keys()

In [ ]:
from LiveAMP import *

A = pd.DataFrame()
A['n'] = [0, 1, 2]
A['d'] = [0, 0 ,2]
# A['err'] = np.clip(A['n'] / A['d'],-1,1)
A['err'] = np.where(A['d']>0, A['n']/A['d'], pd.NA)
# A['err'] = A['n'] / A['d']
# A[np.isinf(A)] = pd.NA
# A = A.prep()
# A[np.isnan(A)|np.isinf(A) ] = pd.NA
# A[np.isinf(A) ] = pd.NA
# A.mean()
# A.dtypes
A.mean()
# A

In [ ]:
self.__dict__.keys()
self.clf[0].Y

In [ ]:
clf = self.clf[0]
clf.

In [ ]:
from sklearn.calibration import calibration_curve, CalibrationDisplay
# plt.plot(*calibration_curve(self.Y_pred['actual'].values, self.Y_pred['proba'].values, n_bins=100))
CalibrationDisplay.from_predictions(self.Y_pred['actual'], self.Y_pred['proba'], n_bins=10)

In [ ]:
dct['mlt'].query("crse_code=='biol1406' & styp_code=='n'")

In [ ]:
843/1.126294, 800/1.06878, 790/1.056075

In [ ]:
for k,v in dct['rslt'].items():
    v.info()

In [ ]:
for clf in dct['clf']:
    delattr(clf,'Z_proc')

In [ ]:
write('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/Y_pred/n/_allcrse/202308/ca37/20cf/19462.pkl', dct)

In [ ]:
dct['clf'][0].__dict__.keys()

In [ ]:
clf = self.clf_lst[0]
S = clf.Y_pred.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
    'actual': y['actual'].sum(),
    'predicted': y['predicted'].sum(),
    'train_score': clf.train_score,
    'test_score': (1 - f1_score(y['actual'], y['predicted'], sample_weight=1+(2*y['actual']-1)*clf.weight))*100,
})).prep()
proj_mask = S.eval(f"pred_code==@self.proj_code")
proj_col = f'{self.proj_code}_projection'
S[proj_col] = S.loc[proj_mask, 'predicted'].squeeze()
S = S[~proj_mask].join(self.mlt.query(f"crse_code==@self.crse_code").squeeze().rename('mlt'))
for k in ['predicted','actual',proj_col]:
    S[k] *= S['mlt']
alpha = 1
S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
S['error'] = S['predicted'] - S['actual']
S['error_pct'] = S['error'] / S['actual'] * 100
S


In [ ]:
self.mlt.query(f"crse_code==@self.crse_code")

In [ ]:
Y.dtypes

In [ ]:
cols = ['_allcrse_cur', self.crse_code+'_cur', self.crse_code]
Y = self.Y.query("crse_code.isin(@cols)").unstack().droplevel(0,1)
# Y = self.Y.query("crse_code.isin(['_allcrse_cur',@self.crse_code+'_cur',@self.crse_code])").unstack().droplevel(0,1)
Z = [X.join(Y).fillna({c:0 for c in cols}).addlevel('crse_code', self.crse_code).addlevel('train_code', self.train_code).addlevel('clf_hash', self.param['clf'][0]).prep(bool=True, cat=True) for X in self.X_proc]
Z[0].dtypes#.missing()

In [ ]:
self.X_proc[0].dtypes

In [ ]:
self.Y

In [ ]:
self.mlt

In [ ]:
self.Y.loc[self.Y.eval("crse_code!='_allcrse_cur' & credit_hr>0")] = 1

# mask = self.Y.eval("crse_code!='_allcrse_cur'")
# self.Y[mask] = self.Y[mask] > 0
self.Y

In [ ]:
self.Y.query(f"pred_code==@self.train_code & crse_code==@self.crse_code")

In [ ]:
self.crse_code
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
self.Y.query(f"crse_code==@self.crse_code & pred_code=={self.train_code}").sum()

# Y = self.Y.query(f"crse_code.isin({cols})").rsindex(['pidm','pred_code','crse_code']).unstack().droplevel(0,1)
# Y = self.Y.query(f"crse_code.isin(@cols)").rsindex(['pidm','pred_code','crse_code']).unstack().droplevel(0,1).fillna(0)
Y = self.Y.query(f"crse_code.isin(['_allcrse_cur',@self.crse_code+'_cur',@self.crse_code])").rsindex(['pidm','pred_code','crse_code']).unstack().droplevel(0,1)
Y

# X = self.X_proc[0]
# Z = X.join(Y, how='left').fillna({c:0 for c in cols}).prep(bool=True)
# Z.dtypes.disp(10000)

In [ ]:
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/137/X_proc/n/ca37/20cf.pkl')
A.keys()
c = 'math1314'
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])

- connection btw text, web
- interactive text bot
- use question frqeuncy to guide adaptive navigation
- breadth + contract structure

In [ ]:
self.Y.groupby('crse_code').max()

In [ ]:
self.mlt.query("crse_code == '_allcrse'").disp(20)

In [ ]:
self.mlt.query("crse_code == '_allcrse'").disp(20)

In [ ]:
agg = lambda y: y.groupby(['styp_code','pred_code','crse_code']).sum().query("styp_code in ('n','r','t')")
Y = self.Y.query(f"pred_code!={self.proj_code} & not crse_code.str.contains('cur')")
Z = self.X[[]].join(Y.rsindex(['pidm','pred_code','crse_code']), how='inner')
N = agg(Y)
D = agg(Z)
self.mlt = N / D
# N.disp(2)
# D.disp(2)
self.mlt.query("crse_code == '_allcrse'").disp(20)


In [ ]:
self.raw_df.vc('styp_code')

In [ ]:
A = self.reg_df['cur'].reset_index()
A['crse_code'] += '_cur'
B = pd.concat([A, self.reg_df['end'].reset_index()])
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
cols
B

In [ ]:
self.__dict__.keys()

In [ ]:
Y = self.reg_df['end']
Z = self.X[[]].join(Y.rsindex(['pidm','pred_code','crse_code']), how='inner')
agg = lambda y: y.query("styp_code in ('n','r','t') & pred_code!=@self.proj_code").groupby(['styp_code','pred_code','crse_code']).sum()
N = agg(Y)
D = agg(Z)
M = N / D
# M = (N / D).prep()
M.dtypes
# M = (N / D).query("crse_code=='_allcrse'")
# pd.isnull(M).sum()
# np.nanmax(M)
# self.X
# Z

In [ ]:
self.X.info()

In [ ]:
# self.reg_df['cur'].info()
for k,v in self.__dict__.items():
    print(k)
    try:
        v.info()
    except:
        try:
            for a,b in v.items():
                print(a)
                b.info()
        except:
            pass

In [ ]:
repl = {'term_code':'pred_code', 'term_desc':'pred_desc'}
R = {stage: (pd.concat([
            self.terms[pred_code][stage].reg
            .rename(columns=repl)
            .set_index(['pidm','levl_code','styp_code','pred_code','crse_code'])
            ['credit_hr']#.unstack()
        for pred_code in self.pred_codes]).copy().prep()
    ) for stage in ['cur','end']}


In [ ]:
Y = pd.concat(R.values())

In [ ]:
Y.info()

In [ ]:
R['cur'].dtypes
# R['cur'].size

In [ ]:
Y.info()

In [ ]:
Y = R['cur'].rename(columns=lambda x:x+'_cur').join(R['end'], how='outer').fillna(0)
print(Y.info())
cols = Y.columns.drop('_allcrse_cur')
Y[cols] = Y[cols] > 0
print(Y.info())
Y = Y.prep(bool=True)
print(Y.info())
# Y.dtypes, Y.size, Y.memory_usage

In [ ]:
cols = Y.columns.drop('_allcrse_cur')
Y[cols] = Y[cols] > 0
Y = Y.prep()
Y.dtypes, Y.size

In [ ]:
cols = Y.columns.drop('_allcrse_cur')
cols
A[cols] = A[cols].astype(float).fillna(0)
cols.remove('_allcrse_cur')
A[cols] = A[cols] > 0

# cols.drop('_allcrse_cur')
# .difference('_allcrse_cur')

In [ ]:
X = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/terms.pkl')
X.keys()
del X['raw_df']
del X['reg_df']
write('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/terms2.pkl', X, overwrite=True)

In [ ]:
from LiveAMP import *
param_dcts['imp']

In [ ]:
dct = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/stack.pkl')
dct[' 50%'].query("crse_code=='_allcrse' & train_code==202308").disp(100)

In [ ]:
from LiveAMP import *
flg = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/flg/139/202408.parq')
adm = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/adm/140/202408.parq')

@pd_ext
def convert(ser, bool=False, cat=False, dtype_backend='numpy_nullable'):
    assert isinstance(ser, pd.Series)
    if pd.api.types.is_string_dtype(ser) or pd.api.types.is_object_dtype(ser):
        ser = ser.astype('string')
        try:
            ser = pd.to_datetime(ser)
        except ValueError:
            try:
                ser = pd.to_numeric(ser, downcast='integer')
            except ValueError:
                pass
    # if pd.api.types.is_integer_dtype(ser):
    #     ser = ser.astype('Int64')
    # if pd.api.types.is_string_dtype(ser):
    #     ser = ser.str.lower().replace('', pd.NA)
    # if bool:
    #     vals = set(ser.dropna().unique())
    #     for L in [['false','true'], [0,1], ['n','y']]:
    #         if vals.issubset(L):
    #             ser = (ser == L[1]).astype('boolean').fillna(False)
    # if cat and pd.api.types.is_string_dtype(ser):
    #     ser = ser.astype('category')
    return ser.convert_dtypes(dtype_backend)


# adm['id'].convert().dtypes#.disp()
# pd.to_numeric(adm['id'], downcast='integer')

adm['id'].sort_values()


# .prep().disp(2)
# adm.merge(flg, how='left', on=['id','term_code_entry','styp_code'])
# dct = run_amp()
# dct = get_stack(140)
# dct = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/stack.pkl')
# dct[' 50%'].query("crse_code=='_allcrse' & train_code==202308").disp(100)

In [ ]:
qry = f"""select * from opeir.admissions_fall2024 where id='D'"""
qry = f"""select * from opeir.admissions_fall2024 where first_name='Jovani' and last_name='Burnett' order by current_date"""
df = db.execute(qry)
df.disp(10)

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/X.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/140/X_proc/n/ca37/20cf.pkl'
X = MyBaseClass().load(fn)

In [ ]:
30/(25*5*4/60)

In [ ]:
dir(X)

In [ ]:
X.__dict__.keys()

In [ ]:
dct = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/143/stack.pkl')
# dct[' 50%'].reset_index().query("crse_code=='_allcrse' & train_code==202308").sort_values(['train_code','pred_code','error_pct']).disp(100)
dct[' 50%'].query("crse_code=='_allcrse' & train_code==202308").sort_index().disp(100)

In [ ]:
# dct[' 50%'].query("crse_code=='_allcrse'").sort_index().reset_index().disp(1000)
# dct[' 50%'].query("crse_code=='_allcrse'").reset_index().sort_values(['crse_code','sty_code','train_code','pred_code','overall_score']).disp(1000)
# dct[' 50%'].reset_index().sort_values(['crse_code','styp_code','train_code','pred_code','overall_score']).disp(1000)
dct = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/141/stack.pkl')
dct[' 50%'].reset_index().sort_values('overall_score').groupby(['crse_code','styp_code','train_code','pred_code']).first()[['trf_hash','imp_hash','clf_hash','202408_projection','error_pct','overall_score']].round(1).disp(1000)

In [ ]:
dct = {'a':7, 'b':13, 'c':5}
max(dct, key=dct.get)

In [ ]:
dct['rslt'][' 50%'].sort_index().disp(100)

In [ ]:
self.Z.vc('styp_code')

In [ ]:
self = dct['amp'][0]
clf_dct = self.param['clf'][2].copy() | {'split_type':'stratified', 'task':'classification', 'verbose':0, 'log_file_name': self.path.with_suffix('.log')}
cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
y = self.Z[cols]
Z = [X.query(f"pred_code==@self.train_code").join(y) for X in self.X_proc]
clf_dct['time_budget'] = 20

In [ ]:
vc = Z[0].vc(self.crse_code)['ct']
r = vc.sum() / vc / 2
r /= r.min()
r

In [ ]:
total_true = Z[0][self.crse_code].sum()
w = 1/2
for i in range(0, 6):
    j = i % len(Z)
    clf = fl.AutoML(**clf_dct)
    clf.fit(Z[i].drop(columns=self.crse_code), Z[i][self.crse_code], sample_weight=1+w*Z[i][self.crse_code])
    P = [clf.predict(z.drop(columns=self.crse_code)).sum() for z in Z]
    total_pred = np.mean(P)
    err_pct = (total_pred - total_true) / total_true * 100
    print(w, err_pct)
    if abs(err_pct) < 0.1:
        break
    w += (2*(err_pct<0)-1)/(2**(j+2))


In [ ]:
w = 2.5
# Z = Z_proc.query(f"pred_code==@self.train_code")
clf = fl.AutoML(**clf_dct)
1 + (w-1) * Z[self.crse_code]
clf.fit(Z.drop(columns=self.crse_code), Z[self.crse_code], sample_weight=1+(w-1)*Z[self.crse_code])

In [ ]:
total_pred = np.mean([clf.predict(X.join(y).drop(columns=self.crse_code)).sum() for X in self.X_proc[1:]])
(total_pred / total_true - 1) * 100

# for X_proc in self.X_proc[1:]:
#     X = X_proc.join(y)
#     total_pred = clf.predict(X.drop(columns=self.crse_code)).sum()
#     total_pred / total_true - 1
#     break


In [ ]:
Y.disp()

In [ ]:
w = 1


In [ ]:
24319/9827-1

In [ ]:
dct['rslt'][' 50%']

In [ ]:
dct['summary'].disp(100)

In [ ]:
from LiveAMP import *
str(pctl(10))

In [ ]:
dct['clf'][0].param['clf']

In [ ]:
dct['summary'].disp(100)

In [ ]:
# len(dct['clf'])
clf = dct['clf'][0]
w = clf.Z[clf.crse_code].value_counts().rename('sample_weight')
w = w.sum() / w / 2#).reset_index()#to_frame()
w
clf.Z['weight'] =clf.Z[clf.crse_code].replace(w)
clf.Z.disp()
# w
# clf.Z.merge(w,
#             on=clf.crse_code,
#             how='left',
#             left_index=True,
# ).disp();
# clf.Z.join(w, on=clf.crse_code)

In [ ]:
dct['median'].sort_index()

In [ ]:
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/144/X.pkl')

In [ ]:
A.keys()
vc = A['Z']['_allcrse'].value_counts()
vc[False] / vc[True]
# 9723*2.5

In [ ]:
def summarize(self, clf):
    S = clf.Y_pred.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
        'actual': y['actual'].sum(),
        'predicted': y['predicted'].sum(),
        'train_score': clf.train_score,
        'test_score': clf.get_params()['metric'].score(y['actual'], y['predicted']) * 100,
    })).prep()
    proj_mask = S.eval(f"pred_code==@self.proj_code")
    proj_col = f'{self.proj_code}_projection'
    S[proj_col] = S.loc[proj_mask, 'predicted'].squeeze()
    S = S[~proj_mask].join(self.mlt[self.crse_code].rename('mlt'))
    for k in ['predicted','actual',proj_col]:
        S[k] *= S['mlt']
    S['overall_score'] = S['train_score'] + S['test_score']
    S['error'] = S['predicted'] - S['actual']
    S['error_pct'] = S['error'] / S['actual'] * 100
    clf.sims = S[[proj_col,'predicted','actual','error','error_pct','overall_score','test_score','train_score']].copy()
    S = clf.sims.droplevel('sim')
    clf.summary = S.groupby(S.index.names).median()
    return clf


In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/144/Y_pred/n/_allcrse/202108/ca37/e843/9fbe.pkl')

In [ ]:
w['clf_lst'][0]#.keys()
w['sims']

In [ ]:
def summarize(self, clf):
    S = clf.Y_pred.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
        'actual': y['actual'].sum(),
        'predicted': y['predicted'].sum(),
        'train_score': clf.train_score,
        'test_score': clf.get_params()['metric'].score(y['actual'], y['predicted']) * 100,
    })).prep()

    proj_mask = S.eval(f"pred_code==@self.proj_code")
    proj_col = f'{self.proj_code}_projection'
    S[proj_col] = S.loc[proj_mask, 'predicted'].squeeze()
    S = S[~proj_mask].join(self.mlt[self.crse_code].rename('mlt'))
    for k in ['predicted','actual',proj_col]:
        S[k] *= S['mlt']
    S['overall_score'] = S['train_score'] + S['test_score']
    S['error'] = S['predicted'] - S['actual']
    S['error_pct'] = S['error'] / S['actual'] * 100
    clf.sims = S[[proj_col,'predicted','actual','error','error_pct','overall_score','test_score','train_score']].copy()
    S = clf.sims.droplevel('sim')
    clf.summary = S.groupby(S.index.names).median()
    return clf

self = dct['clf'][0]
clf = self.clf_lst[0]
S = summarize(self, clf)
S.summary

In [ ]:
S.droplevel('sim')
S.groupby([x for x in S.index.names if x!='sim']).median()

In [ ]:
dct['summary'].disp(100)

In [ ]:
from LiveAMP import *
dct = get_stack()

In [ ]:
dct.keys()

In [ ]:
def get_stack(cycle_day=None):
    cycle_day = (Term(term_code=202408).cycle_date-pd.Timestamp.now()).days+1 if cycle_day is None else cycle_day
    path = AMP(cycle_day=cycle_day).root_path 
    path /= rjust(cycle_day,3,0)
    path /= 'Y_pred'
    print(path)
    L = [MyBaseClass().load(F) for A in path.iterdir() for B in A.iterdir() for C in B.iterdir() for D in C.iterdir() for E in D.iterdir() for F in E.iterdir()]
    # # self.stack = {'clf': [clf for clf in L if clf is not None]}
    # # self.stack |= {k: pd.concat([clf[k] for clf in self.stack['clf']]).prep() for k in ['Y_pred', 'summary']}
get_stack()


In [ ]:
dct |= {k: [clf for clf in dct['clf']] for k in ['Y_pred', 'summary']}
dct['summary']
dct |= {k: pd.concat([clf[k] for clf in dct['clf']]).prep() for k in ['Y_pred', 'summary']}
# write(self.root_path / 'stack.pkl', dct)
dct['summary']

In [ ]:
clf = A['clf'][0]
clf.summary

In [ ]:
from lightgbm import LGBMClassifier
a = LGBMClassifier()
a.fit?

In [ ]:
from LiveAMP import *

param_grds = {
    'trf': {
        'act_equiv': 'passthrough',
        'act_equiv_missing': 'passthrough',
        'admt_code': 'drop',
        'apdc_day': 'passthrough',
        'appl_day': 'drop',
        'birth_day': 'passthrough',
        'camp_code': 'drop',
        'coll_code': 'passthrough',
        'distance': ['passthrough'],#, pwrtrf],
        'fafsa_app': 'drop',
        'finaid_accepted': 'drop',
        'gap_score': 'passthrough',
        'gender': 'passthrough',
        'hs_qrtl': 'passthrough',
        'international': 'passthrough',
        'lgcy': 'passthrough',
        'math': 'passthrough',
        'oriented': 'passthrough',
        'pred_code': 'drop',
        **{f'race_{r}': 'passthrough' for r in ['american_indian','asian','black','pacific','white','hispanic']},
        'reading': 'passthrough',
        'remote': 'passthrough',
        'resd': 'passthrough',
        'schlship_app': 'passthrough',
        'ssb': 'passthrough',
        'styp_code': 'drop',
        'waiver': 'passthrough',
        'writing': 'passthrough',
    },
    'imp': {
        'random_state': 42,
        'datasets': 20,
        'iterations': 10,
        # 'datasets': 2,
        # 'iterations': 1,
        'tune': False,
        # 'tune': [False, True],
    },
    'clf': {
        'seed': 42,
        'metric': F_beta(1),
        'early_stop': True,
        'time_budget': 60,
        'estimator_list': [['lgbm','xgboost']],#,'catboost','histgb','extra_tree','xgb_limitdepth','rf','lrl1','lrl2','kneighbor'
        # 'ensemble': [False, True],
        'ensemble': False,
        'eval_method': 'cv',
        'n_splits': 5,
    },
}

formatter = lambda x: str(x).replace('\n','').replace(' ','')
hasher = lambda x, d=2: hashlib.shake_128(formatter(x).encode()).hexdigest(d)
param_dcts = dict()
for key, val in param_grds.items():
    lst = cartesian(val, sort=True, key=str)
    if key == 'trf':
        lst = [[(c,t,["__"+c]) for c,t in trf.items() if t not in ['drop', None, '']] for trf in lst]
    param_dcts[key] = [[hasher(k), formatter(k), k] for k in lst]

for styp_code in ['n']:
    for crse_code in ['_allcrse']:#,'math1314']:
        for train_code in [202108, 202208, 202308]:
            for param in cartesian(param_dcts):
                self = AMP(
                    cycle_day=(Term(term_code=202408).cycle_date-pd.Timestamp.now()).days+1,
                    styp_code=styp_code,
                    crse_code=crse_code,
                    train_code=train_code,
                    param=param,
                    # cycle_day=145,
                    # crse_code='math1314',
                )
                self.run()
                # assert 1==2

# self.run()
# self.overwrite
# self.mlt

In [ ]:
import flaml as fl
w = fl.AutoML
w.fit?

In [ ]:
G = self.stack()
G[0]

In [ ]:
G[0].keys()

In [ ]:
# G = [MyBaseClass().load(F) for A in (self.root_path / 'Y_pred').iterdir() for B in A.iterdir() for C in B.iterdir() for D in C.iterdir() for E in D.iterdir() for F in E.iterdir() if F.suffix=='.pkl']
G = pd.concat([read(F)['summary'] for A in (self.root_path / 'Y_pred').iterdir() for B in A.iterdir() for C in B.iterdir() for D in C.iterdir() for E in D.iterdir() for F in E.iterdir() if F.suffix=='.pkl'])
# G = [MyBaseClass().load(F) for A in (self.root_path / 'Y_pred').iterdir() for B in A.iterdir() for C in B.iterdir() for D in C.iterdir() for E in D.iterdir() for F in E.iterdir()]
# G[0].summary

In [ ]:
G[0]

In [ ]:
self = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/144/Y_pred/n/_allcrse/202208/ca37/e843/4dfa.pkl')

In [ ]:
self

In [ ]:
for clf in self['clf_lst']:
    S = clf.Y.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
        'actual': y['actual'].sum(),
        'predicted': y['predicted'].sum(),
        'train_score': clf.train_score,
        'test_score': (1 - clf.get_params()['metric'].score(y['actual'], y['predicted'])) * 100,
    })).prep()
    S['error'] = S['predicted'] - S['actual']
    S['error_pct'] = S['error'] / S['actual'] * 100
    clf.S = S.copy()
    S.disp()


In [ ]:
self.X_proc[0].dtypes

In [ ]:
self.Y = self.reg_df['cur'].rename(columns=lambda x:x+'_cur').join(self.reg_df['end'], how='outer')
cols = self.Y.columns.drop('_allcrse_cur')
self.Y[cols] = self.Y[cols] > 0
self.Z = self.X.join(self.Y.rsindex(['pidm','pred_code']), how='left')

In [ ]:
Z = self.Z
# Z.filter(self.reg_df['end'].columns.tolist()).loc[:,'_allcrse':].sum().disp(10000)
Z.loc[:,'_allcrse_cur':'_allcrse'].sum().disp(10000)

In [ ]:
self.Z.loc[:,'_allcrse_cur':].filter(self.reg_df['end'].columns.tolist()).groupby(['styp_code','pred_code']).sum()

In [ ]:
self.reg_df['end'].dtypes

In [ ]:
agg = lambda y: y.filter(self.reg_df['end'].columns).groupby(['styp_code','pred_code']).sum()
N = agg(self.Y)
D = agg(self.Z)
M = (N / D).query("styp_code in ('n','r','t') & pred_code!=@self.proj_code")
M[np.isnan(M) | np.isinf(M)] = pd.NA
self.mlt = M.prep()
self.mlt

In [ ]:
agg = lambda y: y.groupby(['styp_code','pred_code']).sum()
N = agg(self.reg_df['end']).query("styp_code in ('n','r','t') & pred_code!=@self.proj_code")
D = agg(self.X[[]].join(self.reg_df['end'].rsindex(['pidm','pred_code'])))
M = (N / D)
M

In [ ]:
self.mlt

In [ ]:
w = self.mlt.iloc[0,3]
self.mlt == w
self.mlt
np.isnan(self.mlt)
pd.isnull(self.mlt)
mask = np.isnan(self.mlt)
self.mlt[np.isnan(self.mlt)] = pd.NA
self.mlt
# self.mlt.fillna(pd.NA)
# self.mlt.replace(np.nan, pd.NA).prep(bool=True)

In [ ]:
agg = lambda y: y.groupby(['styp_code','pred_code']).sum()
N = agg(self.reg_df['end'])
D = agg(self.X[[]].join(self.reg_df['end'].rsindex(['pidm','pred_code'])))
self.mlt = (N / D).replace(np.inf, pd.NA).query("styp_code in ('n','r','t') & pred_code!=@self.proj_code").prep()
self.mlt


In [ ]:
5.7

In [ ]:
self.reg_df['cur'].vc('styp_code')

In [ ]:
from term import *
import hashlib, miceforest as mf, flaml as fl
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss, fbeta_score
from sklearn import set_config
set_config(transform_output="pandas")
warnings.filterwarnings("ignore", message="Could not infer format, so each element will be parsed individually, falling back to `dateutil`")

################## Metrics that work for FLAML and sklearn ##################
class Metric(MyBaseClass):
    """Parent class for metrics that work with FLAML and sklearn"""
    def __str__(self):
        return self.__repr__()
    def __lt__(self, other):
        return str(self) < str(other)
    def __call__(self, X_val, y_val, estimator, labels, X_train, y_train, weight_val=None, weight_train=None, *args):
        """for FLAML"""
        start = time.time()
        y_pred = estimator.predict(X_val)
        pred_time = (time.time() - start) / len(X_val)
        val_loss = self.score(y_val, y_pred, labels=labels, sample_weight=weight_val)
        y_pred = estimator.predict(X_train)
        train_loss = self.score(y_train, y_pred, labels=labels, sample_weight=weight_train)
        return val_loss, {
            "val_loss": val_loss,
            "train_loss": train_loss,
            "pred_time": pred_time,
        }

class F_beta(Metric):
    """Implements F_beta for FLAML https://scikit-learn.org/stable/modules/generated/sklearn.metrics.fbeta_score.html"""
    def __init__(self, beta):
        self.beta = float(beta)
    def __repr__(self):
        return f"F_{self.beta:.2f}"
    def score(self, y_true, y_pred, **kwargs):
        return 1 - fbeta_score(y_true, y_pred, beta=self.beta, **kwargs)

class Accuracy(Metric):
    def __repr__(self):
        return "accuracy"
    def score(self, y_true, y_pred, **kwargs):
        return 1 - accuracy_score(y_true, y_pred, **kwargs)

class LogLoss(Metric):
    def __repr__(self):
        return "log loss"
    def score(self, y_true, y_pred, **kwargs):
        return log_loss(y_true, y_pred, **kwargs)

################## AMP ##################
@dataclasses.dataclass
class AMP0(MyBaseClass):
    cycle_day : int = 0
    pred_codes: tuple = (202108, 202208, 202308, 202408)
    show: set = dataclasses.field(default_factory=set)
    root_path: str = f"/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/"

    def get_X(self):
        self.terms = {
            pred_code: {
                stage: Term(term_code=pred_code, cycle_day=cycle_day, overwrite=self.overwrite, show=self.show).run()
            for stage, cycle_day in {'cur': self.cycle_day, 'end':0}.items()}
        for pred_code in self.pred_codes}
        repl = {'term_code':'pred_code', 'term_desc':'pred_desc'}
        self.reg_df = {stage: (pd.concat([
                    self.terms[pred_code][stage].reg
                    .rename(columns=repl)
                    .set_index(['pidm','levl_code','styp_code','pred_code','crse_code'])
                    ['credit_hr'].unstack()
                for pred_code in self.pred_codes]).copy().prep()
            ) for stage in ['cur','end']}
        self.Y = self.reg_df['cur'].rename(columns=lambda x:x+'_cur').join(self.reg_df['end'], how='outer')
        cols = self.Y.columns.drop('_allcrse_cur')
        self.Y[cols] = self.Y[cols] > 0

        self.raw_df = pd.concat([
                    self.terms[pred_code]['cur'].raw
                    .rename(columns=repl)
                for pred_code in self.pred_codes]).copy().dropna(axis=1, how='all').reset_index(drop=True).prep()

        R = self.raw_df.copy()
        repl = {'ae':0, 'n1':1, 'n2':2, 'n3':3, 'n4':4, 'r1':1, 'r2':2, 'r3':3, 'r4':4}
        R['hs_qrtl'] = pd.cut(R['hs_pctl'], bins=[-1,25,50,75,90,101], labels=[4,3,2,1,0], right=False).combine_first(R['apdc_code'].map(repl))
        R['remote'] = R['camp_code'] != 's'
        R['resd'] = R['resd_code'] == 'r'
        R['oriented'] = R['oriented'] != 'n'
        R['lgcy'] = ~R['lgcy_code'].isin(['n','o'])
        R['majr_code'] = R['majr_code'].replace({'0000':pd.NA, 'und':pd.NA, 'eled':'eted', 'agri':'unda'})
        R['coll_code'] = R['coll_code'].replace({'ae':'an', 'eh':'ed', 'hs':'hl', 'st':'sm', '00':pd.NA})
        R['coll_desc'] = R['coll_code'].map({
            'an': 'ag & natural_resources',
            'ba': 'business',
            'ed': 'education',
            'en': 'engineering',
            'hl': 'health sciences',
            'la': 'liberal & fine arts',
            'sm': 'science & mathematics',
            pd.NA: 'no college designated',
        })
        majr = ['majr_desc','dept_code','dept_desc','coll_code','coll_desc']
        S = R.sort_values('cycle_date').drop_duplicates(subset='majr_code', keep='last')[['majr_code',*majr]]
        X = R.drop(columns=majr).merge(S, on='majr_code', how='left').prep(bool=True)
        checks = [
            'cycle_day >= 0',
            'apdc_day >= cycle_day',
            'appl_day >= apdc_day',
            'birth_day >= appl_day',
            'birth_day >= 5000',
            'distance >= 0',
            'hs_pctl >=0',
            'hs_pctl <= 100',
            'hs_qrtl >= 0',
            'hs_qrtl <= 4',
            'act_equiv >= 1',
            'act_equiv <= 36',
            'gap_score >= 0',
            'gap_score <= 100',
        ]
        for check in checks:
            mask = X.eval(check)
            assert mask.all(), [check,X[~mask].disp(5)]

        fill = {
            'birth_day': ['median',['pred_code','styp_code']],
            'remote': False,
            'international': False,
            **{f'race_{r}': False for r in ['american_indian','asian','black','pacific','white','hispanic']},
            'lgcy': False,
            'resd': False,
            'waiver': False,
            'fafsa_app': False,
            'schlship_app': False,
            'finaid_accepted': False,
            'ssb': False,
            'math': False,
            'reading': False,
            'writing': False,
            'gap_score': 0,
            'oriented': False,
        }
        for k, v in fill.items():
            X[k] = X.impute(k, *listify(v))
        M = X.isnull().rename(columns=lambda x:x+'_missing')

        code_desc = lambda x: [x+'_code', x+'_desc']
        attr = [
            'index',
            'pidm',
            *code_desc('pred'),
            *code_desc('apdc'),
            *code_desc('levl'),
            *code_desc('styp'),
            *code_desc('admt'),
            *code_desc('camp'),
            *code_desc('coll'),
            *code_desc('dept'),
            *code_desc('majr'),
            *code_desc('cnty'),
            *code_desc('stat'),
            *code_desc('natn'),
            *code_desc('resd'),
            *code_desc('lgcy'),
            'international',
            'gender',
            *[f'race_{r}' for r in ['american_indian','asian','black','pacific','white','hispanic']],
            'waiver',
            'birth_day',
            'distance',
            'hs_qrtl',
        ]
        self.X = X.join(M).reset_index().set_index(attr, drop=False).rename(columns=lambda x:'__'+x).prep(bool=True)



@dataclasses.dataclass
class AMP(MyBaseClass):
    cycle_day : int = 0
    proj_code : int = 202408
    train_code: int = 202308
    pred_codes: tuple = (202108, 202208, 202308, 202408)
    crse_code : str = '_allcrse'
    styp_code : str = 'n'
    show: set = dataclasses.field(default_factory=set)
    param: dict = dataclasses.field(default_factory=dict)    
    root_path: str = f"/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/"
    dependence: dict = dataclasses.field(default_factory=lambda: {
            'adm':'raw', 'flg':'raw', 'dst':'raw', 'raw':'terms', 'reg':'terms', 'terms':{'raw_df','reg_df'}, 'raw_df':'X', 'reg_df':{'Y','mlt'}, 
            'X':{'Y','anon','mlt','X_proc'}, 'X_proc':'Z_proc', 'Y':{'mlt','anon','Z_proc'}, #'Z_proc':'Y_pred',
            'Y_pred':'summary', 'summary':'optimal',
        })
    

    def run(self):
        self.root_path /= rjust(self.cycle_day,3,0)
        self.__dict__ |= AMP0(**{k: self[k] for k in ['cycle_day', 'pred_codes', 'root_path', 'show']}).get(f"X.pkl").__dict__
        self.root_path /= self.styp_code
        self.root_path /= self.crse_code
        self.get("Z.pkl")
        return self

    def get_Z(self):
        qry = f"styp_code==@self.styp_code"
        cols = uniquify(['_allcrse_cur', self.crse_code+'_cur', self.crse_code])
        self.Z = (
            self.X
            .query(qry)
            .join(self.Y.rsindex(['pidm','pred_code'])[cols].astype(float))
            .fillna({k:0 for k in cols})
            .prep(bool=True)
        )
        agg = lambda y: y.groupby(['styp_code','pred_code'])[self.crse_code].sum().rename('mlt').to_frame().query(qry)
        N = agg(self.Y)
        D = agg(self.Z)
        self.mlt = (N / D).replace(np.inf, pd.NA).query("pred_code!=@self.proj_code").prep()

self = AMP(
    train_code=202108,
    cycle_day=(Term(term_code=202408).cycle_date-pd.Timestamp.now()).days+1,
    # cycle_day=145,
    # crse_code='math1314',
)
self.run()

# self.run()

# self = Term(term_code=202208, cycle_day=147).run()
# self = Term(term_code=202208, cycle_day=147)
# self = AMP(cycle_day=Term(term_code=202408)).run()
# /home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/145/data.pkl

In [ ]:
self.mlt

In [ ]:
self.terms[202208]['cur'].raw.dtypes.disp(4)
self.raw_df.dtypes.disp(4)
self.X.dtypes.disp(1000)

In [ ]:
qry = f"styp_code==@self.styp_code"
agg = lambda y: y.groupby(['styp_code','pred_code'])[self.crse_code].sum().rename('mlt').to_frame().query(qry)#+f" & pred_code!=self.proj_code")
N = agg(self.Y)
D = agg(self.Z)
self.mlt = (N / D).replace(np.inf, pd.NA).query("pred_code!=@self.proj_code").prep()
self.mlt


In [ ]:
self.mlt

In [ ]:
# self.Z.reset_index().iloc[:,-2:]#.prep(bool=True)#.sum().disp(10)
# self.mlt
qry = "levl_code=='ug' & styp_code in ('n','r','t')"
where = lambda df: df.query()#.copy()
agg = lambda y: y.groupby(['levl_code','styp_code','pred_code'])[self.crse_code].sum().rename('mlt').to_frame().query(qry+f" & pred_code!={self.proj_code}")

N = agg(self.Y)
N
# D = agg(self.Z)
# self.mlt = (N / D).replace(np.inf, pd.NA).prep()


In [ ]:
self.X.columns.drop('__index')#.difference(['__index'])
# w = self.X.columns
# w.drop('__index')

In [ ]:
self.X

In [ ]:
(
self.reg_df['cur'].rsindex(['pidm','pred_code'])[list({self.crse_code, '_allcrse'})].rename(columns=lambda x:x+'_cur')
# .join(self.reg_df['end'].rindex(['pidm','pred_code'])[self.crse_code])
)

In [ ]:
self.reg_df['end'].rindex(['pidm','pred_code']).disp()

In [ ]:
repl = {'term_code':'pred_code', 'term_desc':'pred_desc'}
R = {stage: (pd.concat([
            self.terms[pred_code][stage].reg
            .rename(columns=repl)
            .query("crse_code==@self.crse_code or crse_code=='_allcrse'")
            .set_index(['pidm','levl_code','styp_code','pred_code','crse_code'])
            ['credit_hr'].unstack()
        for pred_code in self.pred_codes]).copy().prep()
    ) for stage in ['cur','end']}
R['end']
# ['credit_hr'].unstack()

In [ ]:
self.Z.prep(bool=True)
# .vc('math1314')

In [ ]:
self.Z.missing()

In [ ]:
c = self.Z.columns.tolist()
c
c.remove('__index')
c
# dir(c)

In [ ]:
Y = self.reg_df['cur']['credit_hr'].unstack().rename(columns=lambda x:x+'_cur').join(self.reg_df['end']['credit_hr'].unstack(), how='outer')
cols = Y.columns.difference(['_allcrse_cur'])
Y[cols] = Y[cols]>0
Y

In [ ]:
grp = ['levl_code','styp_code','crse_code','pred_code']
# N = self.reg_df['end'].groupby(grp)['credit_hr'].sum()

# N.disp()
self.reg_df['end']

In [ ]:
self.terms[202208]['cur'].reg

In [ ]:
self.reg_df['cur']#.reset_index().unstack()

In [ ]:
self.reg_df['cur'].disp(1)
# self.terms[202308]['cur'].reg.columns

In [ ]:
self.terms[202308]['cur'].reg

In [ ]:
self.terms[202208]['cur']

In [ ]:
select distinct x.CURRENT_DATE, 
count(*) over(partition by x.CURRENT_DATE) as cnt
from opeir.REGISTRATION_FALL2024 x
order by 1 desc

select distinct x.CURRENT_DATE, 
count(*) over(partition by x.CURRENT_DATE) as cnt
from opeir.admissions_fall2024 x
order by 1 desc

In [ ]:
self.raw

In [ ]:
a = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/147/202208.pkl')
a['raw']

In [ ]:
from term import *
self = Term(term_code=202208, cycle_day=147)#.run()
# t = '202208'
# self.trm.query("term_code==@t")
# ser = self.trm[['term_code']].convert()
# ser.convert(
# ser.dtypes
# pd.to_numeric(a, downcast='integer').dtype
# a.prep().dtype
# if pd.api.types.is_string_dtype(ser) or pd.api.types.is_object_dtype(ser):
#     ser = ser.astype('string')
#     ser = pd.to_numeric(ser, downcast='integer')
#     print(ser.dtype)
#     try:
#         ser = pd.to_datetime(ser)
#     except ValueError:
#         try:
#             ser = pd.to_numeric(ser, downcast='integer')
#         except ValueError:
#             pass

# .vc('term_code').disp(100)
# [self.trm.query("term_code==@t").squeeze() for t in self.appl_term_codes]
# [w.trm.query("term_code==@t").squeeze() for t in self.appl_term_codes]

In [ ]:
from common import *
Common().trm

In [ ]:
pd.read_parquet('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/dst.parquet')
# df = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/dst.pkl')
# write('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/data/dst.parquet', df)

In [ ]:
import pathlib
root_path = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/147')
fn = 'name/456.pkl'
fn.split('/')[0].split('.')[0]

In [ ]:
import numpy as np, pandas as pd
def prep(X, cap='casefold', bool=False, cat=False):
    if isinstance(X, str):
        return (getattr(X, cap)() if cap else X).strip()
    elif isinstance(X, (list, tuple, set, pd.Index)):
        return type(X)(prep(x, cap) for x in X)
    elif isinstance(X, dict):
        return {prep(k, cap): prep(v, cap) for k, v in X.items()}
    elif isinstance(X, pd.DataFrame):
        g = lambda x: prep(x, cap).replace(' ','_').replace('-','_') if isinstance(x, str) else x
        X = X.rename(columns=g).rename_axis(index=g)
        idx = pd.MultiIndex.from_frame(X[[]].reset_index().convert(bool=bool, cat=cat))
        return X.convert(bool=bool, cat=cat).set_index(idx).rename_axis(X.index.names)
    elif isinstance(X, pd.Series):
        assert 1==2
    else:
        return X
prep(set([1,'a',3]))

In [ ]:
x = 2
match x%4:
    case 0:
        print('even')
    case 1:
        print('odd')
    case 2 | 3:
        print('odd2')

In [ ]:
import dill
def read(path, overwrite=False, **kwargs):
    if overwrite:
        delete(path)
    try:
        with open(path, 'rb') as f:
            return dill.load(f, **kwargs)
    except:
        return None

path = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/147/False/anon.pkl'
with open(path, 'rb') as f:
    w =  dill.load(f)
g = lambda x: x.upper().replace(' ','_').replace('-','_') if isinstance(x, str) else x
z = w['_allcrse']
# z.rename(g(z.name)).rename_axis(index=g)
z.index

In [ ]:
# cols = intersection([*self.trf_grid.keys(),*code_desc('camp'),*code_desc('coll'),*code_desc('levl'),*code_desc('pred'),*code_desc('styp')])

# Y = self.get('Y')[['_allcrse_cur','_allcrse']]

Y = self.get('Y')
X = self.get('X')[['__'+x for x in self.trf_grid.keys()]]
Z = X.join(Y).rsindex([*code_desc('camp'),*code_desc('coll'),*code_desc('levl'),*code_desc('pred'),*code_desc('styp')])
# Z.missing()
Z
# .rsindex([*code_desc('camp'),*code_desc('coll'),*code_desc('levl'),*code_desc('pred'),*code_desc('styp')])[['__'+x for x in self.trf_grid.keys()]]
# X.disp(2)


In [ ]:
1+1

In [ ]:
class fbeta:
    def __init__(self, beta):
        self.beta = beta
    def __str__(self):
        return f'f{self.beta}'
    def __call__(self, X_val, y_val, estimator, labels, X_train, y_train, weight_val=None, weight_train=None, *args):
        start = time.time()
        y_pred = estimator.predict(X_val)
        pred_time = (time.time() - start) / len(X_val)
        val_loss = 1 - fbeta_score(y_val, y_pred, beta=self.beta, labels=labels, sample_weight=weight_val)
        y_pred = estimator.predict(X_train)
        train_loss = 1 - fbeta_score(y_train, y_pred, beta=self.beta, labels=labels, sample_weight=weight_train)
        return val_loss, {
            "val_loss": val_loss,
            "train_loss": train_loss,
            "pred_time": pred_time,
        }
f = fbeta(2)
formatter = lambda x: str(x).replace('\n','').replace(' ','')
hasher = lambda x, d=2: hashlib.shake_128(str(x).encode()).hexdigest(d)
hasher(formatter(f))

In [ ]:
def wrapper(beta):
    def func(x,y):
        return x+beta*y
    func.__name__ = 'hi'
    func.__str__ = lambda: 'hi'
    return func
f = wrapper(2)
str(f)

def my_function():
    print("This is my function.")

# Setting a custom string representation
my_function.__str__ = lambda: "Custom Description of my_function"

# print(my_function)

class C:
    def __call__(self, x):
        return 2*x
    def __str__(self):
        return 'hi'
f = C()
str(f)

In [ ]:
self.clf_dct

In [ ]:
import seaborn as sns
from LiveAMP import *
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/149/False/X_proc/n/all/all/fa15/9b0f/all.pkl')
A.keys()
X = A['X_proc'][0]
# X[0].describe()
for nm, ser in X.select_dtypes('number').items():
    ser.describe().disp(100)
    sns.kdeplot(ser)
    plt.title(nm)
    plt.show()

In [ ]:
len(self.trf_dct)

In [ ]:
S.sort_values('error_pct').query("imp_hash=='9b0f' & clf_hash in ['1d1f','c244']")#.groupby(['crse_code','levl_code','styp_code','train_code','pred_code']).first()
S.query("imp_hash=='9b0f' & clf_hash in ['1d1f','c244']")

In [ ]:
self.imp_dct

In [ ]:
S = pd.concat([read(F)['summary'] for A in self.path['summary'].iterdir() for B in A.iterdir() for C in B.iterdir() for D in C.iterdir() for E in D.iterdir() for F in E.iterdir()])
A = S.reset_index('sim', drop=True)
A.groupby(A.index.names).median().reset_index().sort_index().query("pred_code==202308")

In [ ]:
self.clf_dct

In [ ]:
S.disp(1000)

In [ ]:
A = S.reset_index('sim', drop=True)
A.groupby(A.index.names).median().reset_index().sort_index()

In [ ]:
S

In [ ]:
self.get('mlt')

In [ ]:
from LiveAMP import *
proj_code = 202408
path = {'styp_code': 'n', 'crse_code': '_allcrse'}
mlt = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/mlt.pkl')
M = mlt.query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']").rsindex('pred_code').squeeze().rename(f'{proj_code}_projection')
A = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Y_pred/'+join(path.values(),'/'))
for B in sorted(A.iterdir()):
    path['train_code'] = B.stem
    for C in sorted(B.iterdir()):
        path['trf_hash'] = C.stem
        for D in sorted(C.iterdir()):
            path['imp_hash'] = D.stem
            for E in sorted(D.iterdir()):
                path['clf_hash'] = E.stem
                dct = read(E)
                for path['sim'], (Y, ts) in enumerate(zip(dct['Y_pred'], dct['train_score'])):
                    print(path)
                    # A = Y.groupby('pred_code')
                    A = Y.addlevel('imp_hash', path['imp_hash']).addlevel('sim', path['sim'])
                    S = A.groupby([*path.keys(),'pred_code']).apply(lambda y: pd.Series({
                        'train_score': ts,
                        'test_score': (1 - f1_score(y['actual'], y['predicted'])) * 100,
                        'actual': y['actual'].sum(),
                        'predicted': y['predicted'].sum(),
                        })).prep()
                    S.insert(0, 'overall_score', S['train_score'] + S['test_score'])
                    S['error'] = S['predicted'] - S['actual']
                    S['error_pct'] = S['error'] / S['actual'] * 100
                    proj_mask = S.eval("pred_code==@proj_code")
                    S.loc[proj_mask, [c for c in S.columns if 'score' in c or 'error' in c]] = pd.NA
                    S = S.join(M*S.loc[proj_mask, 'predicted'].squeeze())
                    S.disp(10)
                    assert 1==2
                    S



In [ ]:
(5127-2731)/2731

In [ ]:
dct = dict()
lst = [1,2]
for dct['a'] in lst:
    print('hi')
dct


In [ ]:
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Z_proc/n/all/all/fa15/9f20/all.pkl')
targ = '_allcrse'
clf_par = {
    'ensemble':True,
    'estimator_list':['lgbm','xgboost','rf','extra_tree','histgb'],
    'metric':'f1',
    'seed':42,
    'split_type':'stratified',
    'task':'classification',
    'time_budget': 60*10,
    'early_stop': True,
    'eval_method': 'cv',
    'n_splits': 5,
    'verbose':0}
cols = uniquify([*A['Z_proc'][0].filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = A['Z_proc'][0].filter(cols).copy()
trn_mask = Z.eval(f"pred_code==202308")
clf = fl.AutoML(**clf_par)
clf.fit(Z[trn_mask].drop(columns=targ), Z[trn_mask][targ], **clf_par, log_file_name=self.root_path / 'log.txt')
# plt.barh(clf.model.estimator.feature_name_, clf.model.estimator.feature_importances_)
# plt.show()
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=120)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=np.inf)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
from LiveAMP import *
A = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/150/False/Z_proc/n/all/all/fa15/9f20/all.pkl')
targ = '_allcrse'
cols = uniquify([*A['Z_proc'][0].filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = A['Z_proc'][0].filter(cols).copy()
trn_mask = Z.eval(f"pred_code==202308")

dt = 30
clf_par = {
    'ensemble':True,
    'estimator_list':[
        'lgbm',
        'xgboost',
        # 'rf',
        # 'extra_tree',
        # 'histgb',
        ],
    'metric':'f1',
    'seed':42,
    'split_type':'stratified',
    'task':'classification',
    'time_budget': 120,
    # 'time_budget': dt,
    'early_stop': True,
    'eval_method': 'cv',
    'n_splits': 5,
    'verbose': 0,
    'log_file_name': '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/log.txt',
    'log_type': 'all',
    'X_train': Z[trn_mask].drop(columns=targ),
    'y_train': Z[trn_mask][targ],
    }

def plot():
    time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=clf_par['log_file_name'], time_budget=np.inf)
    plt.title("Learning Curve")
    plt.xlabel("Wall Clock Time (s)")
    plt.ylabel("Validation Accuracy")
    plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
    plt.show()

clf = fl.AutoML(**clf_par)
clf.fit(**clf_par)
plot()
# for k in range(100):
#     clf_par['time_budget'] += dt
#     clf.retrain_from_log(**{k: clf_par[k] for k in {'log_file_name','X_train','y_train'}})
#     clf.fit(**clf_par)
#     plot()

In [ ]:
Y = (
    Z[targ].rename('actual').to_frame()
    .assign(predicted=clf.predict(Z.drop(columns=targ)))
    .assign(proba=clf.predict_proba(Z.drop(columns=targ))[:,1])
)
Y

In [ ]:
clf.predict(Z)

In [ ]:
clf.predict(Z.drop(columns=targ))

In [ ]:
Y = (
    Z[targ].rename('actual').to_frame()
    .assign(predicted=clf.predict(Z.drop(columns=targ)))
    .assign(proba=clf.predict_proba(Z.drop(columns=targ))[:,1])
)


In [ ]:
q = fl.AutoML()
q.fit?

In [ ]:
clf.retrain_from_log?

In [ ]:
clf.retrain_from_log

In [ ]:
vars(clf)

In [ ]:
plt.barh(clf.model.estimator.feature_name_, clf.model.estimator.feature_importances_)

In [ ]:
from flaml.automl.data import get_output_from_log

import matplotlib.pyplot as plt
import numpy as np

time_history, best_valid_loss_history, valid_loss_history, config_history, metric_history = fl.automl.data.get_output_from_log(filename=self.root_path / "log.txt", time_budget=120)
plt.title("Learning Curve")
plt.xlabel("Wall Clock Time (s)")
plt.ylabel("Validation Accuracy")
plt.step(time_history, 1 - np.array(best_valid_loss_history), where="post")
plt.show()

In [ ]:
path = {'nm': 'Z_proc', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}
dct = deepcopy(self.get(path | {'nm':'Z_proc', 'crse_code':'all', 'train_code':'all', 'clf_hash':'all'})) | {'Y_pred':[], 'train_score': []}
targ = path['crse_code']
# df = dct['Z_proc']
clf_par = self.clf_dict[path['clf_hash']]
clf_par['seed'] = self.random_state

clf_par['time_budget'] = 2
clf_par['estimator_list'] = ['lgbm', 'xgboost', 'xgb_limitdepth', 'rf', 'extra_tree']

# for imp, Z in enumerate(df):
# Z = df[0]
for Z in deepcopy(dct['Z_proc']):
    cols = uniquify([*Z.filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
    Z = Z.filter(cols).copy().addlevel('crse_code', targ).addlevel('train_code', path['train_code']).addlevel('clf_hash', path['clf_hash']).prep().prep_category()
    trn_mask = Z.eval(f"pred_code==@path['train_code']")
    clf = fl.AutoML(**clf_par)
    # with warnings.catch_warnings(action='ignore'):
    clf.fit(X_train=Z[trn_mask].drop(columns=targ), y_train=Z[trn_mask][targ], **clf_par)
    print('done')
        #  eval_method='cv', n_splits=self.n_splits, **clf_par)

In [ ]:
{'nm': 'Z_proc', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}: {'nm': 'Y_pred', 'styp_code': 'n', 'crse_code': '_allcrse', 'train_code': 202308, 'trf_hash': '8dec', 'imp_hash': '2438', 'clf_hash': '0213'}
path = dct['path'] |{'clf_hash': '5023', 'crse_code':'_allcrse', 'train_code':202308}
targ = path['crse_code']
clf_par = self.clf_dict[path['clf_hash']]
clf_par['seed'] = self.random_state
clf_par['time_budget'] = 2
clf_par['ensemble'] = {"final_estimator": LogisticRegression()}
print(clf_par)
Z = dct['Z_proc'][0]
cols = uniquify([*Z.filter(like='__').columns, '_allcrse_cur', targ+'_cur', targ])
Z = Z.filter(cols).copy().addlevel('crse_code', targ).addlevel('train_code', path['train_code']).addlevel('clf_hash', path['clf_hash']).prep().prep_category()
trn_mask = Z.eval(f"pred_code==@path['train_code']")
clf = fl.AutoML(**clf_par)
with warnings.catch_warnings(action='ignore'):
    clf.fit(Z[trn_mask].drop(columns=targ), Z[trn_mask][targ], eval_method='cv', n_splits=self.n_splits, **clf_par)

# Z

In [ ]:
fl.AutoML?

In [ ]:
X.groupby(['pred_code'])['__fafsa_app'].mean()

In [ ]:
self.trf_dict

In [ ]:
dct['holdout_score']

In [ ]:
hasher = lambda t: hashlib.shake_128(str(t).encode()).hexdigest(2)
for nm in ['trf','imp','clf']:
    print(nm)
    self[nm+'_list'] = cartesian({k: uniquify(v, key=str) for k,v in self[nm+'_grid'].items()})
    print(len(self[nm+'_list']))
    if nm == 'trf':
        self[nm+'_list'] = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items() if t not in ['drop',None,'']], remainder='drop', verbose_feature_names_out=False) for trf in self[nm+'_list']]
        self[nm+'_dict'] = {hasher(t.transformers): t for t in self[nm+'_list']}
    else:
        self[nm+'_dict'] = {hasher(t): t for t in self[nm+'_list']}
    print(len(self[nm+'_dict']))


In [ ]:

hashlib.shake_128(str(x.transformers).encode()).hexdigest(2)

In [ ]:
for x in self['trf_list']:
    print(str(x).replace('\n','').replace(' ',''))

In [ ]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/53c7/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/5afa/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/b051/c519/5742.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/b314/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/e69c/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/e858/c519/5742.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/151/False/Y_pred/n/math1314/202308/ed94/c519/5742.pkl'
A = read(fn)
A.keys()
len(A['train_score'])

In [ ]:
1+1

In [ ]:
for k,v in self.trf_dict.items():
    print(k)
    print(str(v).replace('\n','').replace(' ',''))

In [ ]:
A = self.optimal['n']['_allcrse'][202308]
a = min(A, key=lambda x: x['score']['agg'])
a.keys()

# [0].keys()

In [ ]:
for path in cartesian({'nm': '', 'styp_code': self.styp_codes, 'crse_code': self.crse_codes, 'train_code': self.train_codes}, sort=False):
    s = self.get(path | {'nm': 'summary'})
s['53c7']['3d79']['5742'].keys()

In [ ]:
cartesian({'styp_code': self.styp_codes, 'crse_code': self.crse_codes, 'train_code': self.train_codes, 'trf_hash': self.trf_dict.keys(), 'imp_hash': self.imp_dict.keys(), 'clf_hash': self.clf_dict.keys()})


In [ ]:
self.terms[202108]['cur'].get('raw')

In [ ]:
self.get('reg_df')

In [ ]:
self.terms

In [ ]:
cartesian({'pred_code': self.pred_codes})

In [ ]:
import codetiming
x = 'hi'
with codetiming.Timer(text=x+"Elapsed time: {:0.4f} seconds"):
    for i in range(100000):
        pass

In [ ]:
from LiveAMP import *

# FLAGS().run()
self = AMP(**kwargs)
self.get('raw_df')
self.get('reg_df')
self.get('X')
self.get('Y')
self.get('mlt')
for styp_code in self.styp_codes:
    for crse_code in self.crse_codes:
        for train_code in self.train_codes:
            for trf_hash in self.trf_dict.keys():
                for imp_hash in self.imp_dict.keys():
                    for clf_hash in self.clf_dict.keys():
                        path = {'nm':'summary', 'styp_code': styp_code, 'crse_code':crse_code, 'train_code':train_code, 'trf_hash':trf_hash, 'imp_hash':imp_hash, 'clf_hash':clf_hash}
                        print(path)
                        dct = self.get(path)

In [ ]:
dct['rslt'].describe().T

In [ ]:
clf['Y_pred'][0]
clf.keys()
clf['2024_projection_202108']
clf.keys()
pd.DataFrame({k:v for k,v in clf.items() if 'score' in k or 'projection' in k})
# for k in filter(lambda k,v: 'score' in k, clf.items()):
#     print(k)

In [ ]:
def f(dct):
    dct.pop('a')

class A():
    def __init__(self):
        dct = {'a':3}
        f(dct)
        print(dct)
A()

In [ ]:
M = self.get('mlt').query(f"styp_code=='n' & crse_code=='_allcrse'").rsindex('pred_code').squeeze()
M
{k:v for k,v in M.items()}


In [ ]:
clf['rslt'].describe().T.disp(10)

In [ ]:
self.get('X').dtypes.disp(100)

In [ ]:
M = self.get('mlt').query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']")
A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).reset_index()[['pred_code','mlt']]
A.to_dict('records')
# ('index')

In [ ]:
M = self.get('mlt').query(f"styp_code==@path['styp_code'] & crse_code==@path['crse_code']")
M
# A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).reset_index()[['pred_code','mlt']]
# A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).rindex('pred_code', bare=True).sindex('pred_code')#.values
A = (clf['Y_pred'].query('imp==0')['predicted'].sum() * M).rsindex('pred_code').squeeze().to_dict()
# {a['pred_code']: a['mlt'] for a in A}
# A.to_dict('index')
# # ('index')

In [ ]:
self.get('mlt').reset_index().dtypes

In [ ]:
clf['Y_pred'].prep().join(self.get('mlt').prep()).disp(4)

In [ ]:
import seaborn as sns
sns.kdeplot(clf['rslt']['projection21'])
clf['rslt']['projection21'].describe()

In [ ]:
self.mlt.disp(100)

In [ ]:
clf['rslt']

In [ ]:
1+1

In [ ]:
imp.keys()
# imp

In [ ]:
path = {'nm':'test', 'sub':'test2'}
L = listify(path)
nm = L.pop(0)
(self.root_path / join(L,'/')).with_suffix('.pkl')
path

In [ ]:
4200/10/20

In [ ]:
X_imp.disp(1)

In [ ]:
12800/9
17.95*20*4
21*20*52

In [ ]:
str(trf).replace('\n','').replace(' ','')

In [ ]:
import hashlib
self.trf_list = cartesian({k: uniquify(v, key=str) for k,v in self.trf_grid.items()})
self.trf_list = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items() if t not in ['drop',None,'']], remainder='drop', verbose_feature_names_out=False) for trf in self.trf_list]
self.trf_dict = {hashlib.md5(str(trf).encode()).hexdigest(): trf for trf in self.trf_list}
self.trf_dict


# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]
# self.trf_list = [ColumnTransformer([(c,t,["__"+c]) for c,t in trf.items()], remainder='drop', verbose_feature_names_out=False) for trf in self.trf_list]
# self.trf_dict = {hashlib.md5(trf).encode()).hexdigest(): trf for trf in self.trf_list}
# self.trf_dict
# trf_par = self.trf_list[0]
# trf = ColumnTransformer([(c,t,["__"+c]) for c,t in trf_par.items()], remainder='drop', verbose_feature_names_out=False)
# hashlib.md5(str(trf).encode()).hexdigest()
# str(trf).replace('\n','').replace(' ','')
# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]

# w = self.trf_list[0]
# json.dumps(w)
# w = [str({str(k):str(v) for k,v in t.items()}) for t in self.trf_list][0]
# print(w)
# print(hash(w))
# self.trf_list = {hash(stringify(t)): t for t in self.trf_list}
# self.trf_list
# {hash(str(t)):1 for t in self.trf_list}
# self.trf_list[0]
# self.trf_list = [uniquify({k:v for k,v in t.items() if v not in ['drop',None,'']}) for t in self.trf_list]


In [ ]:
# hash(stringify(self.trf_list[0]))
# trf = StandardScaler()
# hash(trf)
hash(str(self.trf_list[0]))

In [ ]:
self.trf_list

In [ ]:
self.get('raw_df')
self.get('reg_df')['cur']
# self.get('X')
# self.get('Y')
# .set_index(['pidm','pred_code','crse_code'])

In [ ]:
from flags import *

@dataclasses.dataclass
class TERM(MyBaseClass):
    cycle_day: int = 0
    term_code: int = 202308
    root_path: str = root_path
    show: set = dataclasses.field(default_factory=set)


    def __post_init__(self):
        self.dependence: dict = {'adm':'raw', 'flg':'raw', 'dst':'raw', 'raw':'term', 'reg':'term'}
        super().__post_init__()
    #     pass
        # print(self.__dict__())
TERM(overwrite={'adm'}).overwrite

In [ ]:
dependence = {'adm':'raw', 'flg':'raw', 'dst':'raw', 'raw':'term', 'reg':'term'}
l = 0
overwrite = {'adm'}
while l < len(overwrite):
    l = len(overwrite)
    overwrite |= {y for x in overwrite for y in setify(dependence[x] if x in dependence else {})}
    # print(self.overwrite)
overwrite

In [ ]:
q = TERM(term_code=202408, overwrite={'adm'})
q.overwrite

In [ ]:
a = {'a'}
a |= {'b'}
a

In [ ]:
dep = {
    'a': {'b','c'},
    'b': {'d','e'},
}
over = {'a'}
l = 0
while l < len(over):
    l = len(over)
    over |= {y for x in over for y in setify(dep[x] if x in dep else {})}
    # if s == len(over):
    #     break
    # s = len(over)
over

In [ ]:
def g(L, D):
    I = intersection(L, D.keys())
    g()
    {g(D[y]) for y in intersection(L, D.keys())}



In [ ]:
a = pathlib.Path('/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/src/dst')
a = pathlib.Path(a)

In [ ]:
from setup import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/admitted_matriculation_predictor/src/dst'
dst = read(fn+'.parq')
write(fn+'.pkl', dst)
# fn.w
# .with_suffix('.pkl')